# pyiron - Create structures

This section gives a brief introduction about the various tools available in pyiron to construct atomic structures. For the sake of compatibility, our structure class is written to be compatible with the popular Atomistic Simulation Environment package ([ASE](https://wiki.fysik.dtu.dk/ase/)). We use the [NGLview](http://nglviewer.org/nglview/latest/api.html) package to visualize the structures and trajectories

As preparation for the following discussion we import a few python modules and create a project. 

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
from pyiron.project import Project

pr = Project('structures')

## Bulk crystals

In this section we discuss various possibilities to create bulk crystal structures.

### Command: *create_structure* 

**Signature**: create_structure(element, bravais_basis, lattice_constant)

 + **element**: *str* name of the chemical element as in 'Al'
 + **bravais_basis**: *str* bravais_basis as in 'fcc', 'bcc'
 + **lattice_constant**: *float* lattice constant in Angstrom

Note: The output gives a cubic cell rather than the smallest unit cell.


In [2]:
basis = pr.create_structure('Al', bravais_basis='fcc', lattice_constant=4.05)
basis.plot3d()

NGLWidget()

### Command: *create_ase_bulk* 

**Signature**: pr.create_ase_bulk(name, crystalstructure=None, a=None, c=None, covera=None, u=None, orthorhombic=False, cubic=False)

Creating bulk systems using ASE bulk module:
> https://wiki.fysik.dtu.dk/ase/ase/build/build.html#ase.build.bulk.

Crystal structure and lattice constant(s) will be guessed if not
provided.

+ **name**: *str*
    Chemical symbol or symbols as in 'MgO' or 'NaCl'.
+ **crystalstructure**: *str*
    Must be one of sc, fcc, bcc, hcp, diamond, zincblende,
    rocksalt, cesiumchloride, fluorite or wurtzite.
+ **a**: *float*
    Lattice constant.
+ **c**: *float*
    Lattice constant.
+ **c_over_a**: *float*
    c/a ratio used for hcp.  Default is ideal ratio: sqrt(8/3).
+ **u**: *float*
    Internal coordinate for Wurtzite structure.
+ **orthorhombic**: *bool*
    Construct orthorhombic unit cell instead of primitive cell
    which is the default.
+ **cubic**: *bool*
    Construct cubic unit cell if possible.


**Example:** fcc bulk aluminum in a cubic cell

In [3]:
basis = pr.create_ase_bulk('Al', cubic=True)
basis.plot3d()

NGLWidget()

**Example:** wurtzite GaN in a 3x3x3 repeated orthoromic cell.

Note: In contrast to new_basis = basis.repeat() which creates a new object, set_repeat() modifies the existing structure object.

In [4]:
basis = pr.create_ase_bulk('AlN', crystalstructure='wurtzite', a=3.5, orthorhombic=True)
basis.set_repeat([3,3,3])
basis.plot3d()

NGLWidget()

### Spacegroup (use ase crystal command)

In [6]:
from ase.spacegroup import crystal
from pyiron_atomistics.structure.atoms import ase_to_pyiron

a = 9.04
skutterudite = crystal(('Co', 'Sb'),
                       basis=[(0.25, 0.25, 0.25), (0.0, 0.335, 0.158)],
                       spacegroup=204,
                       cellpar=[a, a, a, 90, 90, 90])
skutterudite = ase_to_pyiron(skutterudite)

Comments (**TODO**): The present realization is not really pyiron-like. We should either use the previous schema to replace in ASE our atom and atoms class or we could extend the project object:
+ pr.create_ase.crystal()
+ or: pr.convert.from_ase(ase_structure) # having an easy to access converter may be generally useful 

In [7]:
skutterudite.plot3d()

NGLWidget()

## Point defects

### Vacancy

In [8]:
basis = pr.create_ase_bulk('Al', cubic=True)
basis.set_repeat([4,4,4])
del basis[0]
basis.plot3d()

NGLWidget()

Random set of n$_{vac}$ vacancies

In [9]:
n_vac = 24
basis = pr.create_ase_bulk('Al', cubic=True)
basis.set_repeat([4,4,4])
del basis[np.random.permutation(len(basis))[:n_vac]]
print (len(basis))
basis.plot3d()

232


NGLWidget()

### Random substitutial alloys

In [10]:
n_sub = 24
basis = pr.create_ase_bulk('Al', cubic=True)
basis.set_repeat([4,4,4])
basis[np.random.permutation(len(basis))[:n_sub]] = 'Mg'
print (len(basis))
basis.plot3d()

256


NGLWidget()